In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ścieżka do głównego folderu z danymi
base_dir = r"C:\Users\matik\Documents\Data Sets\Faulty_solar_panel"

# Przygotowanie generatorów danych (treningowych i walidacyjnych)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    base_dir, 
    target_size=(224, 224),  # Rozmiar wejściowy dla ResNet50
    batch_size=32,
    class_mode='categorical',  # Klasyfikacja wieloklasowa
    subset='training'  # Używamy 80% danych do treningu
)

valid_gen = datagen.flow_from_directory(
    base_dir, 
    target_size=(224, 224),  # Rozmiar wejściowy dla ResNet50
    batch_size=32,
    class_mode='categorical',  # Klasyfikacja wieloklasowa
    subset='validation'  # Używamy 20% danych do walidacji
)


Found 711 images belonging to 6 classes.
Found 174 images belonging to 6 classes.


In [13]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

# Ładowanie wstępnie wytrenowanego modelu ResNet50 bez głowy (bo dodamy swoje warstwy)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Zamrażanie warstw bazowych (żeby nie trenować ich ponownie)
base_model.trainable = False

# Budowanie modelu
model = models.Sequential([
    base_model,  # Wstępnie wytrenowany model ResNet50
    layers.GlobalAveragePooling2D(),  # Agregacja cech w wektorze
    layers.Dense(512, activation='relu'),  # Warstwa gęsta
    layers.Dense(train_gen.num_classes, activation='softmax')  # Wyjście z klasyfikacją
])

# Kompilacja modelu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Podsumowanie modelu
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 512)               1049088   
                                                                 
 dense_7 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 24,639,878
Trainable params: 1,052,166
Non-trainable params: 23,587,712
_________________________________________________________________


In [14]:
# Trenowanie modelu
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // train_gen.batch_size,
    epochs=25,
    validation_data=valid_gen,
    validation_steps=valid_gen.samples // valid_gen.batch_size
)


Epoch 1/25
22/22 [==============================] - 39s 2s/step - loss: 1.8053 - accuracy: 0.2445 - val_loss: 1.5641 - val_accuracy: 0.3125
Epoch 2/25
22/22 [==============================] - 35s 2s/step - loss: 1.5428 - accuracy: 0.3932 - val_loss: 1.5459 - val_accuracy: 0.3438
Epoch 3/25
22/22 [==============================] - 33s 2s/step - loss: 1.4789 - accuracy: 0.4300 - val_loss: 1.3828 - val_accuracy: 0.5063
Epoch 4/25
22/22 [==============================] - 33s 2s/step - loss: 1.3804 - accuracy: 0.4993 - val_loss: 1.3859 - val_accuracy: 0.4375
Epoch 5/25
22/22 [==============================] - 34s 2s/step - loss: 1.3737 - accuracy: 0.4757 - val_loss: 1.5406 - val_accuracy: 0.3750
Epoch 6/25
22/22 [==============================] - 33s 2s/step - loss: 1.3322 - accuracy: 0.4683 - val_loss: 1.3982 - val_accuracy: 0.4437
Epoch 7/25
22/22 [==============================] - 34s 2s/step - loss: 1.3121 - accuracy: 0.4934 - val_loss: 1.2780 - val_accuracy: 0.5437
Epoch 8/25
22/22 [==

In [11]:
# Ocena modelu
loss, accuracy = model.evaluate(valid_gen)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


6/6 [==============================] - 7s 1s/step - loss: 1.2521 - accuracy: 0.4655
Loss: 1.252073884010315
Accuracy: 0.4655172526836395


In [ ]:
import matplotlib.pyplot as plt

# Rysowanie wykresów dla straty
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Rysowanie wykresów dla dokładności
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
model.save('resnet50_solar_fault_detection_model.h5')